## Injection benchmark

CPython 3.6.5 and an i7 7700K were used for the timings

In [1]:
from antidote import antidote
import attr

world = antidote.container

In [2]:
@antidote.register
class Service1:
    pass


@antidote.register
class Service2:
    def __init__(self, service1: Service1):
        self.service1 = service1
       
 
@antidote.register
class Service3:
    def __init__(self, service1: Service1, service2: Service2):
        self.service1 = service1
        self.service2 = service2

  
@antidote.register
class Service4:
    def __init__(self, service1: Service1, service2: Service2, service3: Service3):
        self.service1 = service1
        self.service2 = service2
        self.service3 = service3


### Function call

Injection overhead is here measured with a function which does nothing.

In [3]:
def f(s1: Service1, s2: Service2, s3: Service3, s4: Service4):
    pass

Time necessary to only execute the function, without retrieving the services

In [4]:
args = (world[Service1], world[Service2], world[Service3], world[Service4])
%timeit f(*args)

77.9 ns ± 1.14 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Overhead of the injection when all argument must be retrieved from the container.

In [5]:
f_injected = antidote.inject(f)
%timeit f_injected()

1.59 µs ± 7.15 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Overhead of the injection when no argument has to be retrieved.

In [6]:
%timeit f_injected(*args)

571 ns ± 3.37 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


If speed is critical, arguments can be bound with a `functools.partial`.

In [7]:
f_bound = antidote.inject(f, bind=True)

%timeit f_bound()

216 ns ± 1.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


It should be noted that this is the worst scenario possible. In a real case example, the function would be much slower.
To put those results into perspective, the overhead is roughly the time needed to decode this simple JSON.

In [8]:
import json
%timeit json.loads('{ "name":"John", "age":30, "city":"New York"}')

2.15 µs ± 32.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Object instantiation

A similar benchmark is done with object instantiation.

In [9]:
class Obj:
    s1: Service1
    s2: Service2
    s3: Service3
    s4: Service4
        
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit Obj(*args)

291 ns ± 2.66 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [10]:
class ObjInjected:
    s1: Service1
    s2: Service2
    s3: Service3
    s4: Service4
        
    @antidote.inject
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit ObjInjected()

1.84 µs ± 17.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [11]:
@attr.s
class ObjAttrs:
    s1: Service1 = antidote.attrib()
    s2: Service2 = antidote.attrib()
    s3: Service3 = antidote.attrib()
    s4: Service4 = antidote.attrib()
        
%timeit ObjAttrs()

1.51 µs ± 5.68 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Conclusion

The key take away from those benchmarks, is to avoid using injection on short functions which are called repeatedly, in a loop typically. In the other cases, the overhead is considered to be negligeable.

Yet should it not be the case, when measured with a profiling tool, consider to either use the option `bind=True` or to retrieve the necessary services beforehand.